# Imports

In [1]:
import os
from collections import OrderedDict
from io import StringIO

import anndata
import numpy as np
import pandas as pd
import scanpy

scanpy.settings.verbosity = 3
scanpy.logging.print_versions()

pd.options.display.max_columns = 100

outdir_gdrive = '/home/olga/googledrive/TabulaMicrocebus/data/cross-species'
outdir_local = '/home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/cross-species'
outdirs = outdir_gdrive, outdir_local


Bad key "text.kerning_factor" on line 4 in
/home/olga/miniconda3/envs/tabula-microcebus-v2/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


-----
anndata     0.7.5
scanpy      1.6.0
sinfo       0.3.1
-----
PIL                 8.1.0
anndata             0.7.5
backcall            0.1.0
cairo               1.18.2
cffi                1.13.2
cloudpickle         1.2.2
colorama            0.4.4
cycler              0.10.0
cython_runtime      NA
dask                2.9.0
dateutil            2.8.0
decorator           4.4.1
get_version         2.1
h5py                2.10.0
igraph              0.7.1
importlib_metadata  1.3.0
ipykernel           5.1.3
ipython_genutils    0.2.0
ipywidgets          7.5.1
jedi                0.15.1
joblib              0.14.1
kiwisolver          1.1.0
legacy_api_wrap     1.2
leidenalg           0.7.0
llvmlite            0.30.0
louvain             0.6.1
matplotlib          3.1.2
more_itertools      NA
mpl_toolkits        NA
natsort             6.2.0
numba               0.46.0
numexpr             2.7.0
numpy               1.17.3
packaging           20.8
pandas              1.1.4
parso               0.5.1
pex

In [12]:
kmer_outdir = '/home/olga/data_lg/data_sm_copy/immune-evolution/data-objects/human-mouse/'
kmer_outdir

'/home/olga/data_lg/data_sm_copy/immune-evolution/data-objects/human-mouse/'

## Def Describe

In [2]:
def describe(df, random=False):
    print(df.shape)
    print("--- First 5 entries ---")
    display(df.head())
    if random:
        print('--- Random subset ---')
        display(df.sample(5))

# Load orthology from MGI

In [4]:
! wget http://www.informatics.jax.org/downloads/reports/HOM_MouseHumanSequence.rpt 

--2021-01-11 14:43:30--  http://www.informatics.jax.org/downloads/reports/HOM_MouseHumanSequence.rpt
Resolving www.informatics.jax.org (www.informatics.jax.org)... 64.147.54.32
Connecting to www.informatics.jax.org (www.informatics.jax.org)|64.147.54.32|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8379875 (8.0M)
Saving to: ‘HOM_MouseHumanSequence.rpt’

HOM_MouseHumanSeque 100%[===================>]   7.99M  7.53MB/s    in 1.1s    

2021-01-11 14:43:32 (7.53 MB/s) - ‘HOM_MouseHumanSequence.rpt’ saved [8379875/8379875]

--2021-01-11 14:43:32--  http://../data/HOM_MouseHumanSequence.rpt
Resolving .. (..)... failed: Name or service not known.
wget: unable to resolve host address ‘..’
FINISHED --2021-01-11 14:43:32--
Total wall clock time: 1.3s
Downloaded: 1 files, 8.0M in 1.1s (7.53 MB/s)


In [3]:
mgi_orthology = pd.read_csv('http://www.informatics.jax.org/downloads/reports/HOM_MouseHumanSequence.rpt', sep='\t')
describe(mgi_orthology)

(40015, 13)
--- First 5 entries ---


,HomoloGene ID,Common Organism Name,NCBI Taxon ID,Symbol,EntrezGene ID,Mouse MGI ID,HGNC ID,OMIM Gene ID,Genetic Location,"Genomic Coordinates (mouse: , human: )",Nucleotide RefSeq IDs,Protein RefSeq IDs,SWISS_PROT IDs
0,3,"mouse, laboratory",10090,Acadm,11364,MGI:87867,NaN,NaN,Chr3 78.77 cM,Chr3:153922357-153944632(-),NM_007382,NP_031408,P45952
1,3,human,9606,ACADM,34,NaN,HGNC:89,OMIM:607008,Chr1 p31.1,Chr1:75724347-75763679(+),"NM_001286042,NM_001286044,NM_000016,NM_0011273...","NP_001120800,NP_001272971,NP_001272973,NP_0000...",P11310
2,5,"mouse, laboratory",10090,Acadvl,11370,MGI:895149,NaN,NaN,Chr11 42.96 cM,Chr11:70010183-70015411(-),NM_017366,NP_059062,P50544
3,5,human,9606,ACADVL,37,NaN,HGNC:92,OMIM:609575,Chr17 p13.1,Chr17:7217125-7225267(+),"NM_001270448,NM_001270447,NM_001033859,NM_000018","XP_006721579,XP_024306509,XP_011522132,XP_0115...",P49748
4,6,"mouse, laboratory",10090,Acat1,110446,MGI:87870,NaN,NaN,Chr9 29.12 cM,Chr9:53580522-53610350(-),NM_144784,NP_659033,Q8QZT1


## Get only 1:1 orthologs

In [7]:
mgi_one2one = mgi_orthology.groupby("HomoloGene ID").filter(
    lambda x: len(x) == 2 and len(x["Common Organism Name"].unique()) == 2
)
describe(mgi_one2one)

(32934, 13)
--- First 5 entries ---


,HomoloGene ID,Common Organism Name,NCBI Taxon ID,Symbol,EntrezGene ID,Mouse MGI ID,HGNC ID,OMIM Gene ID,Genetic Location,"Genomic Coordinates (mouse: , human: )",Nucleotide RefSeq IDs,Protein RefSeq IDs,SWISS_PROT IDs
0,3,"mouse, laboratory",10090,Acadm,11364,MGI:87867,NaN,NaN,Chr3 78.77 cM,Chr3:153922357-153944632(-),NM_007382,NP_031408,P45952
1,3,human,9606,ACADM,34,NaN,HGNC:89,OMIM:607008,Chr1 p31.1,Chr1:75724347-75763679(+),"NM_001286042,NM_001286044,NM_000016,NM_0011273...","NP_001120800,NP_001272971,NP_001272973,NP_0000...",P11310
2,5,"mouse, laboratory",10090,Acadvl,11370,MGI:895149,NaN,NaN,Chr11 42.96 cM,Chr11:70010183-70015411(-),NM_017366,NP_059062,P50544
3,5,human,9606,ACADVL,37,NaN,HGNC:92,OMIM:609575,Chr17 p13.1,Chr17:7217125-7225267(+),"NM_001270448,NM_001270447,NM_001033859,NM_000018","XP_006721579,XP_024306509,XP_011522132,XP_0115...",P49748
4,6,"mouse, laboratory",10090,Acat1,110446,MGI:87870,NaN,NaN,Chr9 29.12 cM,Chr9:53580522-53610350(-),NM_144784,NP_659033,Q8QZT1


In [8]:
mgi_one2one_2d = mgi_one2one.pivot(index='HomoloGene ID', values='Symbol', columns='Common Organism Name')
describe(mgi_one2one_2d)

(16467, 2)
--- First 5 entries ---


Common Organism Name,human,"mouse, laboratory"
HomoloGene ID,,
3,ACADM,Acadm
5,ACADVL,Acadvl
6,ACAT1,Acat1
7,ACVR1,Acvr1
9,SGCA,Sgca


# Mouse: Tabula Muris Senis

## Load into Scanpy

In [9]:
%%time

h5ad = os.path.join(
    "/home/olga/data_lg/data_sm_copy/czb-tabula-muris-senis/Data-objects/",
    "tabula-muris-senis-droplet-official-raw-obj--no-duplicate-barcodes-per-seq-run--minimal-obs-unified-celltypes.h5ad",
)
mouse = scanpy.read_h5ad(h5ad)
mouse

CPU times: user 1.34 s, sys: 3.2 s, total: 4.53 s
Wall time: 34.9 s


AnnData object with n_obs × n_vars = 238915 × 20138
    obs: 'age', 'cell_id', 'cell_ontology_class', 'cell_ontology_id', 'free_annotation', 'individual', 'n_genes', 'sex', 'subtissue', 'tissue', 'species', 'species_latin', 'channel', 'sequencing_run', 'cell_barcode', 'n_counts', 'narrow_group', 'broad_group', 'compartment_group'
    var: 'n_cells'

## Subset on 1:1 orthologs

In [10]:
gene_mask = mouse.var.index.isin(
    mgi_one2one_2d["mouse, laboratory"].values
)
print(gene_mask.sum())
mouse_one2one = mouse[:, gene_mask]
mouse_one2one

14325


View of AnnData object with n_obs × n_vars = 238915 × 14325
    obs: 'age', 'cell_id', 'cell_ontology_class', 'cell_ontology_id', 'free_annotation', 'individual', 'n_genes', 'sex', 'subtissue', 'tissue', 'species', 'species_latin', 'channel', 'sequencing_run', 'cell_barcode', 'n_counts', 'narrow_group', 'broad_group', 'compartment_group'
    var: 'n_cells'

In [24]:
mouse_one2one.var.head()

,n_cells
index,
Xkr4,147
Rp1,233
Sox17,27289
Mrpl15,95312
Lypla1,86096


## Set human genes gene namees

In [28]:
human_to_mouse_symbol = mgi_one2one_2d.set_index('mouse, laboratory')['human']
describe(human_to_mouse_symbol)

(16467,)
--- First 5 entries ---


mouse, laboratory
Acadm      ACADM
Acadvl    ACADVL
Acat1      ACAT1
Acvr1      ACVR1
Sgca        SGCA
Name: human, dtype: object

In [36]:
mouse_new_var = mouse_one2one.var.reset_index()
mouse_new_var = mouse_new_var.rename(columns={'index': 'mouse_gene_symbol'})
mouse_new_var['human_gene_symbol'] = mouse_new_var.mouse_gene_symbol.map(human_to_mouse_symbol)
mouse_new_var = mouse_new_var.set_index('human_gene_symbol')
describe(mouse_new_var)

(14325, 2)
--- First 5 entries ---


,mouse_gene_symbol,n_cells
human_gene_symbol,,
XKR4,Xkr4,147
RP1,Rp1,233
SOX17,Sox17,27289
MRPL15,Mrpl15,95312
LYPLA1,Lypla1,86096


## Add new var

In [37]:
mouse_one2one.var = mouse_new_var
describe(mouse_one2one.var)

(14325, 2)
--- First 5 entries ---


,mouse_gene_symbol,n_cells
human_gene_symbol,,
XKR4,Xkr4,147
RP1,Rp1,233
SOX17,Sox17,27289
MRPL15,Mrpl15,95312
LYPLA1,Lypla1,86096


## Write 1:1 orthologs

In [38]:
%%time

h5ad = os.path.join(
    kmer_outdir, "tabula-muris-senis--human-mouse-one2one-ortholog-subset.h5ad"
)
print(h5ad)
mouse_one2one.write(h5ad)

/home/olga/data_lg/data_sm_copy/immune-evolution/data-objects/human-mouse/tabula-muris-senis--human-mouse-one2one-ortholog-subset.h5ad
CPU times: user 4.06 s, sys: 1.95 s, total: 6.02 s
Wall time: 34.2 s


# Human: Tabula Sapiens

In [11]:
%%time

h5ad = os.path.join('/home/olga/data_lg/data_sm_copy/tabula-sapiens/data-objects/Pilot1_Pilot2_decontX_Oct2020__minimal_no_pilot2_diaphragm.h5ad')
sapiens = scanpy.read(h5ad, cache=True)
sapiens

AnnData object with n_obs × n_vars = 132884 × 58870
    obs: 'individual', 'tissue', 'subtissue', 'cell_barcode', 'channel', 'sex', 'n_counts', 'n_genes', 'cell_ontology_class', 'species', 'species_latin', 'narrow_group', 'broad_group', 'compartment_group'
    var: 'ensembl_id', 'len'
    uns: 'Propagated.Annotationcollapsed_colors', '_scvi', 'annotation_oct2020_colors', 'cell_ontology_type_colors', 'compartment_colors', 'compartment_oct2020_colors', 'donor_colors', 'leiden', 'method_colors', 'neighbors', 'umap'
    obsm: 'X_scvi', 'X_scvi_decontX', 'X_scvi_raw', 'X_umap', 'X_umap_decontX', 'X_umap_raw'
    layers: 'raw_counts'
    obsp: 'connectivities', 'distances'

## Subset to 1:1 orthologs

In [14]:
gene_mask = sapiens.var.index.isin(mgi_one2one_2d["human"].values)
print(gene_mask.sum())

# Need to copy so it's not an ArrayView object
sapiens_one2one = sapiens[:, gene_mask].copy()
sapiens_one2one

16156


AnnData object with n_obs × n_vars = 132884 × 16156
    obs: 'individual', 'tissue', 'subtissue', 'cell_barcode', 'channel', 'sex', 'n_counts', 'n_genes', 'cell_ontology_class', 'species', 'species_latin', 'narrow_group', 'broad_group', 'compartment_group'
    var: 'ensembl_id', 'len'
    uns: 'Propagated.Annotationcollapsed_colors', '_scvi', 'annotation_oct2020_colors', 'cell_ontology_type_colors', 'compartment_colors', 'compartment_oct2020_colors', 'donor_colors', 'leiden', 'method_colors', 'neighbors', 'umap'
    obsm: 'X_scvi', 'X_scvi_decontX', 'X_scvi_raw', 'X_umap', 'X_umap_decontX', 'X_umap_raw'
    layers: 'raw_counts'
    obsp: 'connectivities', 'distances'

In [17]:
sapiens_one2one.var.head()

,ensembl_id,len
SAMD11,ENSG00000187634.12,1699.705882
NOC2L,ENSG00000188976.11,1743.500000
KLHL17,ENSG00000187961.14,1380.000000
PLEKHN1,ENSG00000187583.10,1643.000000
ISG15,ENSG00000187608.9,698.000000


## Write sapiens subset to file

In [18]:
h5ad = os.path.join(
    kmer_outdir, "tabula-sapiens--human-mouse-one2one-ortholog-subset.h5ad"
)
sapiens_one2one.write(h5ad)


# Concatenate objects

In [39]:
human_mouse = sapiens_one2one.concatenate(
    mouse_one2one,
    batch_key="species_batch",
    batch_categories=["human", "mouse"],
)
human_mouse

AnnData object with n_obs × n_vars = 371799 × 14245
    obs: 'individual', 'tissue', 'subtissue', 'cell_barcode', 'channel', 'sex', 'n_counts', 'n_genes', 'cell_ontology_class', 'species', 'species_latin', 'narrow_group', 'broad_group', 'compartment_group', 'age', 'cell_id', 'cell_ontology_id', 'free_annotation', 'sequencing_run', 'species_batch'
    var: 'ensembl_id-human', 'len-human', 'mouse_gene_symbol-mouse', 'n_cells-mouse'

## Add cell id columnn

In [45]:
human_mouse.obs['cell_id'] = human_mouse.obs.channel.astype(str) + '__' + human_mouse.obs.cell_barcode.astype(str)
describe(human_mouse.obs)

(371799, 20)
--- First 5 entries ---


,individual,tissue,subtissue,cell_barcode,channel,sex,n_counts,n_genes,cell_ontology_class,species,species_latin,narrow_group,broad_group,compartment_group,age,cell_id,cell_ontology_id,free_annotation,sequencing_run,species_batch
index,,,,,,,,,,,,,,,,,,,,
AAACCCAGTGTAGCAG_TSP1_bladder_1-0-human,TSP1,Bladder,nan,AAACCCAGTGTAGCAG,TSP1_bladder_1,female,5522.0,1908.0,macrophage,Human,Homo sapiens,macrophage,macrophage,myeloid,nan,TSP1_bladder_1__AAACCCAGTGTAGCAG,nan,nan,nan,human
AAACCCATCAAGTCTG_TSP1_bladder_1-0-human,TSP1,Bladder,nan,AAACCCATCAAGTCTG,TSP1_bladder_1,female,3829.0,1473.0,macrophage,Human,Homo sapiens,macrophage,macrophage,myeloid,nan,TSP1_bladder_1__AAACCCATCAAGTCTG,nan,nan,nan,human
AAACCCATCCTTCGAC_TSP1_bladder_1-0-human,TSP1,Bladder,nan,AAACCCATCCTTCGAC,TSP1_bladder_1,female,6434.0,2062.0,macrophage,Human,Homo sapiens,macrophage,macrophage,myeloid,nan,TSP1_bladder_1__AAACCCATCCTTCGAC,nan,nan,nan,human
AAACCCATCGAAGGAC_TSP1_bladder_1-0-human,TSP1,Bladder,nan,AAACCCATCGAAGGAC,TSP1_bladder_1,female,12273.0,2751.0,macrophage,Human,Homo sapiens,macrophage,macrophage,myeloid,nan,TSP1_bladder_1__AAACCCATCGAAGGAC,nan,nan,nan,human
AAACGAAAGCTCCGAC_TSP1_bladder_1-0-human,TSP1,Bladder,nan,AAACGAAAGCTCCGAC,TSP1_bladder_1,female,4724.0,651.0,plasma cell,Human,Homo sapiens,plasma cell,plasma cell,lymphoid,nan,TSP1_bladder_1__AAACGAAAGCTCCGAC,nan,nan,nan,human


In [47]:
new_obs = human_mouse.obs.reset_index().set_index('cell_id')
new_obs = new_obs.rename(columns={'index': 'original_cell_id'})
describe(new_obs)

(371799, 20)
--- First 5 entries ---


,original_cell_id,individual,tissue,subtissue,cell_barcode,channel,sex,n_counts,n_genes,cell_ontology_class,species,species_latin,narrow_group,broad_group,compartment_group,age,cell_ontology_id,free_annotation,sequencing_run,species_batch
cell_id,,,,,,,,,,,,,,,,,,,,
TSP1_bladder_1__AAACCCAGTGTAGCAG,AAACCCAGTGTAGCAG_TSP1_bladder_1-0-human,TSP1,Bladder,nan,AAACCCAGTGTAGCAG,TSP1_bladder_1,female,5522.0,1908.0,macrophage,Human,Homo sapiens,macrophage,macrophage,myeloid,nan,nan,nan,nan,human
TSP1_bladder_1__AAACCCATCAAGTCTG,AAACCCATCAAGTCTG_TSP1_bladder_1-0-human,TSP1,Bladder,nan,AAACCCATCAAGTCTG,TSP1_bladder_1,female,3829.0,1473.0,macrophage,Human,Homo sapiens,macrophage,macrophage,myeloid,nan,nan,nan,nan,human
TSP1_bladder_1__AAACCCATCCTTCGAC,AAACCCATCCTTCGAC_TSP1_bladder_1-0-human,TSP1,Bladder,nan,AAACCCATCCTTCGAC,TSP1_bladder_1,female,6434.0,2062.0,macrophage,Human,Homo sapiens,macrophage,macrophage,myeloid,nan,nan,nan,nan,human
TSP1_bladder_1__AAACCCATCGAAGGAC,AAACCCATCGAAGGAC_TSP1_bladder_1-0-human,TSP1,Bladder,nan,AAACCCATCGAAGGAC,TSP1_bladder_1,female,12273.0,2751.0,macrophage,Human,Homo sapiens,macrophage,macrophage,myeloid,nan,nan,nan,nan,human
TSP1_bladder_1__AAACGAAAGCTCCGAC,AAACGAAAGCTCCGAC_TSP1_bladder_1-0-human,TSP1,Bladder,nan,AAACGAAAGCTCCGAC,TSP1_bladder_1,female,4724.0,651.0,plasma cell,Human,Homo sapiens,plasma cell,plasma cell,lymphoid,nan,nan,nan,nan,human


## Set new obs

In [48]:
human_mouse.obs = new_obs

## Write to file

In [49]:
%%time

h5ad = os.path.join(
    kmer_outdir, "human-mouse-one2one-orthologs.h5ad"
)
human_mouse.write(h5ad)

# Make per-tissue subsets for selected tissues

In [50]:
tissues = "Bladder", "Lung", "Muscle"

In [51]:
human_mouse_subset = human_mouse[human_mouse.obs.tissue.isin(tissues)]
human_mouse_subset

View of AnnData object with n_obs × n_vars = 95239 × 14245
    obs: 'original_cell_id', 'individual', 'tissue', 'subtissue', 'cell_barcode', 'channel', 'sex', 'n_counts', 'n_genes', 'cell_ontology_class', 'species', 'species_latin', 'narrow_group', 'broad_group', 'compartment_group', 'age', 'cell_ontology_id', 'free_annotation', 'sequencing_run', 'species_batch'
    var: 'ensembl_id-human', 'len-human', 'mouse_gene_symbol-mouse', 'n_cells-mouse'

In [52]:
for tissue in tissues:
    print(f"--- {tissue} ---")
    tissue_subset = human_mouse_subset[human_mouse_subset.obs.tissue == tissue]
    print(tissue_subset)
    for d in outdirs:
        h5ad = os.path.join(
            kmer_outdir, f"human-mouse-one2one-orthologs--{tissue.lower()}.h5ad"
        )
        print(h5ad)
        %time tissue_subset.write(h5ad)

--- Lung ---
View of AnnData object with n_obs × n_vars = 54919 × 14245
    obs: 'original_cell_id', 'individual', 'tissue', 'subtissue', 'cell_barcode', 'channel', 'sex', 'n_counts', 'n_genes', 'cell_ontology_class', 'species', 'species_latin', 'narrow_group', 'broad_group', 'compartment_group', 'age', 'cell_ontology_id', 'free_annotation', 'sequencing_run', 'species_batch'
    var: 'ensembl_id-human', 'len-human', 'mouse_gene_symbol-mouse', 'n_cells-mouse'
/home/olga/data_lg/data_sm_copy/immune-evolution/data-objects/human-mouse/human-mouse-one2one-orthologs--lung.h5ad
CPU times: user 1.8 s, sys: 587 ms, total: 2.38 s
Wall time: 9.8 s
/home/olga/data_lg/data_sm_copy/immune-evolution/data-objects/human-mouse/human-mouse-one2one-orthologs--lung.h5ad
CPU times: user 1.77 s, sys: 628 ms, total: 2.4 s
Wall time: 9.51 s
--- Muscle ---
View of AnnData object with n_obs × n_vars = 12974 × 14245
    obs: 'original_cell_id', 'individual', 'tissue', 'subtissue', 'cell_barcode', 'channel', 'sex'